<font color="64D7F5">--Comprobación de GPU y versiones--</font>

In [ ]:
# Comprobar GPU y versiones
import torch, platform
print("PyTorch:", torch.__version__) # Versión de PyTorch
print("Python:", platform.python_version()) # Versión de Python
print("GPU disponible:", torch.cuda.is_available()) # Disponibilidad de GPU
if torch.cuda.is_available():
    print("Nombre de la GPU:", torch.cuda.get_device_name(0)) # Nombre de la GPU

<font color="64D7F5">--Instalación de modulos--

In [ ]:
# Instalación de Kaggle
!pip install kaggle

<font color="64D7F5">--Configuración del entorno de trabajo y Dataset(Kaggle)--

In [ ]:
# Subir tu kaggle.json desde tu cuenta de Kaggle
from google.colab import files
files.upload() # selecciona tu kaggle.json

# Crear carpeta para credenciales
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar el Dataset
!kaggle datasets download -d puneet6060/intel-image-classification -p /content/intel-image-classification
!unzip /content/intel-image-classification/intel-image-classification.zip -d /content/intel-image-classification

# Verificar estructura de directorios del Dataset
!find /content/intel-image-classification/ -type d -maxdepth 3

<font color="64D7F5">--Importar librerias necesarias--              
--Definir PATHs--

In [ ]:
# Importando librerias necesarias
from pathlib import Path
import os, random
import numpy as np
import torch

# Definiendo los PATHs sobre cada parte del Dataset
root = Path("/content/intel-image-classification") # Directorio raiz
train_dir = root / "seg_train"/"seg_train" # Directorio de entrenamiento
test_dir = root / "seg_test"/"seg_test" # Directorio de prueba


<font color="64D7F5">--Definiendo semilla de la aleatoriedad--

In [ ]:
# Fijar la semilla para reproducibilidad
Seed = 12
random.seed(Seed); np.random.seed(Seed); torch.manual_seed(Seed); torch.cuda.manual_seed_all(Seed)

<font color="64D7F5">--Conteo de imagenes por clase--

In [ ]:
# Definiendo función para el conteo de imagenes por clase
def count_images_by_class(folder):
    counts = {} # Diccionario para contar imágenes por clase
    for cls in sorted(os.listdir(folder)): # Iterar sobre las clases
        p = os.path.join(folder, cls) # Ruta del directorio de la clase
        if os.path.isdir(p): # Verificar si es un directorio
            n = sum(1 for f in os.listdir(p) if f.lower().endswith((".jpg",".jpeg",".png"))) # Contar imágenes
            counts[cls] = n # Almacenar conteo en el diccionario
    total = sum(counts.values()) # Total de imágenes
    return counts, total # Retornar conteos y total

In [ ]:
#Conteo de imagenes por sección
train_counts, train_total = count_images_by_class(str(train_dir)) #Entrenamiento
test_counts, test_total = count_images_by_class(str(test_dir)) #Prueba

#Mostrando el resultado
print("Clases (train):", list(train_counts.keys()))
print("Imágenes train por clase:", train_counts)
print("TOTAL train:", train_total)
print("TOTAL test:", test_total)



<font color="64D7F5">--Aplicando DataAugmentation(ImageNet)--               
<font color="64D7F5">--Técnicas Clásicas--

In [ ]:
# Definiendo las transformaciones de datos
from torchvision import transforms, datasets
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

img_size = 224 # Tamaño de las imágenes
imagenet_mean = [0.485, 0.456, 0.406] # Media de ImageNet
imagenet_std = [0.229, 0.224, 0.225] # Desviación estándar de ImageNet

# Definiendo las transformaciones de datos para el conjunto de entrenamiento
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)), # Recorte aleatorio y redimensionado
    transforms.RandomHorizontalFlip(), # Flip horizontal aleatorio
    transforms.RandomVerticalFlip(p=0.1), # Flip vertical aleatorio
    transforms.RandomRotation(15), # Rotación aleatoria
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1), scale=(0.9,1.1)), # Transformación afín aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Variación de color
    AutoAugment(AutoAugmentPolicy.IMAGENET), # Aumento de imagen ImageNet
    transforms.ToTensor(), # Conversión a tensor
    transforms.Normalize(imagenet_mean, imagenet_std), # Normalización
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.25)) # Borrado aleatorio
])

# Definiendo las transformaciones de datos para el conjunto de evaluación
eval_tfms = transforms.Compose([
    transforms.Resize(int(img_size * 1.14)), # Redimensionamiento
    transforms.CenterCrop(img_size), # Recorte central
    transforms.ToTensor(), # Conversión a tensor
    transforms.Normalize(imagenet_mean, imagenet_std) # Normalización
])

# Creamos un ImageFolder "base" que usaremos para extraer labels de forma estratificada
full_train_for_labels = datasets.ImageFolder(str(train_dir)) # Dataset completo para entrenamiento
class_to_idx = full_train_for_labels.class_to_idx # Mapeo de clases a índices
idx_to_class = {v: k for k, v in class_to_idx.items()} # Mapeo de índices a clases
num_classes = len(class_to_idx) # Número de clases
num_classes, class_to_idx # Número de clases y mapeo de clases a índices

<font color="64D7F5">--Aplicando DataAugmentation(MixUp)--               
<font color="64D7F5">--Técnicas Modernas--

In [ ]:
# Función para aplicar MixUp
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha) # Parámetro de mezcla
    batch_size = x.size()[0] # Tamaño del batch
    index = torch.randperm(batch_size).to(x.device) # Índices aleatorios

    mixed_x = lam * x + (1 - lam) * x[index, :] # Mezcla de imágenes
    y_a, y_b = y, y[index] # Etiquetas originales y mezcladas
    return mixed_x, y_a, y_b, lam # Retornamos las imágenes y etiquetas mezcladas

# Función para calcular la pérdida de MixUp
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b) # Pérdida de MixUp

<font color="64D7F5">--División del Dataset de entrenamiento en entrenamiento y validación--

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset

targets = [y for (_, y) in full_train_for_labels.samples] # labels alineadas con el orden interno

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=Seed) # División estratificada
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets)) # Índices de entrenamiento y validación

# OJO: hay que usar el MISMO orden de archivos para los dos datasets (transform distintos)
full_train_with_aug = datasets.ImageFolder(str(train_dir), transform=train_tfms) # Dataset completo con aumentos
full_train_eval = datasets.ImageFolder(str(train_dir), transform=eval_tfms) # Dataset completo para evaluación

# Creamos los subsets para entrenamiento y validación
train_ds = Subset(full_train_with_aug, train_idx) # Subconjunto de entrenamiento
val_ds = Subset(full_train_eval, val_idx) # Subconjunto de validación

# Creamos el subset para el conjunto de prueba
test_ds = datasets.ImageFolder(str(test_dir), transform=eval_tfms)

# Mostramos el resultado
print("Train:", len(train_ds), "Val:", len(val_ds), "Test:", len(test_ds))

<font color="64D7F5">--DataLoaders (batching, workers, pin_memory)--

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64 # Tamaño del batch
num_workers = 2 # Número de trabajadores(en Colab suele ir bien 2)
pin_memory = torch.cuda.is_available() # Pin memory para acelerar la transferencia de datos

# Creamos los DataLoaders para el conjunto de entrenamiento
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

# Creamos el DataLoader para el conjunto de validación
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

# Creamos el DataLoader para el conjunto de prueba
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

# Mostramos el resultado
print("Batches - Lotes:\n")
print("Train:", len(train_loader), "Val:", len(val_loader), "Test:", len(test_loader))
print("Classes:", idx_to_class)

<font color="64D7F5">--Vista rápida de un batch (sanity check)--

In [ ]:
import matplotlib.pyplot as plt
import torchvision
import numpy as np

# Visualizamos algunas imágenes del conjunto de entrenamiento
plt.figure(figsize=(10, 10)) # Tamaño de la figura

imgs, labels = next(iter(train_loader)) # Obtenemos un batch de imágenes y etiquetas
grid = torchvision.utils.make_grid(imgs[:32], nrow=8, padding=2) # Creamos una cuadrícula de imágenes

# Des-normalizamos para visualizar
npimg = grid.numpy() # Convertimos a numpy
mean = np.array(imagenet_mean)[:, None, None] # Creamos un array de media
std = np.array(imagenet_std)[:, None, None] # Creamos un array de desviación estándar
npimg = (std * npimg) + mean # Des-normalizamos
npimg = np.clip(npimg, 0, 1) # Clampeamos a [0, 1]

plt.imshow(np.transpose(npimg, (1, 2, 0))) # Mostramos la imagen
plt.axis("off") # Ocultamos los ejes
print("Etiquetas:", [idx_to_class[int(l)] for l in labels[:32]]) # Mostramos las etiquetas
plt.show() # Mostramos la figura

<font color="64D7F5">--Definición de los modelos SimpleCNN, ResNet18 y EfficientNet-B0--

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

# Construimos una CNN simple
def build_simpleCNN(num_classes):
    class SimpleCNN(nn.Module): # Definimos la arquitectura de la red
        def __init__(self, num_classes): # Inicializamos la red
            super(SimpleCNN, self).__init__() # Llamamos al constructor de la clase base
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # Capa convolucional 1
            self.bn1 = nn.BatchNorm2d(32) # Capa de normalización 1
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # Capa convolucional 2
            self.bn2 = nn.BatchNorm2d(64) # Capa de normalización 2
            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # Capa convolucional 3
            self.bn3 = nn.BatchNorm2d(128) # Capa de normalización 3
            self.pool = nn.MaxPool2d(2, 2) # Capa de max pooling
            self.fc1 = nn.Linear(128 * (img_size//8) * (img_size//8), 256) # Capa totalmente conectada 1
            self.fc2 = nn.Linear(256, num_classes) # Capa totalmente conectada 2
            self.dropout = nn.Dropout(0.5) # Capa de dropout

        # Propagación hacia adelante
        def forward(self, x):
            x = self.pool(F.relu(self.bn1(self.conv1(x)))) # 224 -> 112
            x = self.pool(F.relu(self.bn2(self.conv2(x)))) # 112 -> 56
            x = self.pool(F.relu(self.bn3(self.conv3(x)))) # 56 -> 28
            x = x.view(x.size(0), -1) # aplanar
            x = F.relu(self.fc1(x)) # capa totalmente conectada 1
            x = self.dropout(x) # capa de dropout
            x = self.fc2(x) # capa totalmente conectada 2
            return x # capa de salida
    # Fin de la clase
    return SimpleCNN(num_classes)


# ResNet18 preentrenado
def build_ResNet18(num_classes, fine_tune=False): # Construcción de la arquitectura ResNet18
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1) # Cargar modelo preentrenado
    if not fine_tune: # Congelar capas
        for param in model.parameters(): # Congelar todos los parámetros
            param.requires_grad = False # Congelar parámetros
    else: # Descongelar capas específicas
        for name, param in model.named_parameters(): # Iterar sobre los parámetros nombrados
            if "layer4" in name or "fc" in name: # Descongelar capas específicas
                param.requires_grad = True # Descongelar parámetros
            else:
                param.requires_grad = False # Congelar parámetros

    # Ajustar la capa final para el número de clases
    in_features = model.fc.in_features # Obtener el número de características de entrada
    # Reemplazar la capa totalmente conectada
    model.fc = nn.Sequential( # Nueva capa totalmente conectada
        nn.Linear(in_features, 256), # Capa totalmente conectada 1
        nn.ReLU(), # Capa de activación 1
        nn.Dropout(0.5), # Capa de dropout
        nn.Linear(256, num_classes) # Capa totalmente conectada 2
    )
    return model # Modelo ajustado

# EfficientNet-B0 preentrenado
def build_EfficientNetB0(num_classes, fine_tune=False):
    from torchvision import models
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1) # Cargar modelo preentrenado

    if not fine_tune: # Congelar capas
        for param in model.parameters(): # Congelar todos los parámetros
            param.requires_grad = False # Congelar parámetros
    else: # Solo fine-tune en las últimas capas
        for name, param in model.named_parameters(): # Iterar sobre los parámetros nombrados
            if "features.6" in name or "features.7" in name or "classifier" in name: # Descongelar capas específicas
                param.requires_grad = True # Descongelar parámetros
            else:
                param.requires_grad = False # Congelar parámetros

    # Ajustar la capa final para el número de clases
    in_features = model.classifier[1].in_features # Obtener el número de características de entrada
    # Reemplazar la capa totalmente conectada
    model.classifier = nn.Sequential( # Nueva capa totalmente conectada
        nn.Linear(in_features, 256), # Capa totalmente conectada
        nn.ReLU(), # Capa de activación
        nn.Dropout(p=0.4), # Capa de dropout
        nn.Linear(256, num_classes) # Capa totalmente conectada
    )
    return model # Modelo ajustado

<font color="64D7F5">--Loop de entrenamiento y validación--

In [ ]:
# Definiendo una función para el entrenamiento de una época
def train_one_epoch(model, loader, optimizer, criterion, device, use_mixup=True):
    model.train() # Establecer el modelo en modo de entrenamiento
    running_loss, correct, total = 0.0, 0, 0 # Inicializar métricas

    for imgs, labels in loader: # Iterar sobre el DataLoader
        imgs, labels = imgs.to(device), labels.to(device) # Mover imágenes y etiquetas al dispositivo
        optimizer.zero_grad() # Limpiar gradientes

        if use_mixup: # Aplicar MixUp
            imgs, targets_a, targets_b, lam = mixup_data(imgs, labels) # Aplicar MixUp
            outputs = model(imgs) # Propagación hacia adelante
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam) # Calcular la pérdida de MixUp
            _, preds = outputs.max(1) # Obtener las predicciones
            correct += (lam * preds.eq(targets_a).sum().item() + (1 - lam) * preds.eq(targets_b).sum().item()) # Acumular aciertos

        else: # No aplicar MixUp
            outputs = model(imgs) # Propagación hacia adelante
            loss = criterion(outputs, labels) # Calcular pérdida
            _, preds = outputs.max(1) # Obtener las predicciones
            correct += preds.eq(labels).sum().item() # Acumular aciertos

        loss.backward() # Retropropagación
        optimizer.step() # Actualizar pesos

        running_loss += loss.item() * imgs.size(0) # Acumular pérdida
        total += labels.size(0) # Acumular total

    # Retornar métricas
    return running_loss/total, correct/total

# Definir la función de evaluación
def evaluate(model, loader, criterion, device):
    model.eval() # Establecer el modelo en modo de evaluación
    running_loss, correct, total = 0.0, 0, 0 # Inicializar métricas
    with torch.no_grad(): # Desactivar el cálculo de gradientes
        for imgs, labels in loader: # Iterar sobre el DataLoader
            imgs, labels = imgs.to(device), labels.to(device) # Mover imágenes y etiquetas al dispositivo
            outputs = model(imgs) # Propagación hacia adelante
            loss = criterion(outputs, labels) # Calcular la pérdida

            running_loss += loss.item() * imgs.size(0) # Acumular pérdida
            _, preds = outputs.max(1) # Obtener las predicciones
            correct += preds.eq(labels).sum().item() # Acumular aciertos
            total += labels.size(0) # Acumular total

    # Retornar métricas
    return running_loss/total, correct/total

<font color="64D7F5">--Seleccionar modelo a entrenar--

In [ ]:
# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir el tipo de modelo
type_model = "EfficientNetB0" # "SimpleCNN", "ResNet18" o "EfficientNetB0"

# Construir el modelo de acuerdo al tipo seleccionado
if type_model == "SimpleCNN":
  model = build_simpleCNN(num_classes).to(device)
elif type_model == "ResNet18":
  model = build_ResNet18(num_classes).to(device)
elif type_model == "EfficientNetB0":
  model = build_EfficientNetB0(num_classes, fine_tune=True).to(device)

<font color="64D7F5">--Definir pérdida y optimizador--

In [ ]:
import torch.optim as optim

# Definir el optimizador de acuerdo al tipo de modelo
if type_model == "SimpleCNN":
  optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4) # Optimizador para SimpleCNN
else: #ResNet18 / EfficientNetB0
  optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4) # Optimizador para ResNet18 y EfficientNetB0

# Definir la función de pérdida y el programador de tasa de aprendizaje
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

<font color="64D7F5">--Entrenamiento del modelo (varias épocas)--

In [ ]:
import copy


epochs = 30 # Número de épocas
best_acc = 0.0 # Mejor precisión
best_model_wts = copy.deepcopy(model.state_dict()) # Mejor pesos del modelo

patience = 3 # Paciencia para early stopping
counter = 0 # Contador para early stopping

# Historial de métricas
history = {
      'train_loss': [], # Pérdida de entrenamiento
      'train_acc': [], # Precisión de entrenamiento
      'val_loss': [], # Pérdida de validación
      'val_acc': [] # Precisión de validación
}


def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs, best_acc, history):
  # Entrenamiento
  for epoch in range(epochs): # Iterar sobre las épocas
      train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device) # Entrenamiento de una época
      val_loss, val_acc = evaluate(model, val_loader, criterion, device) # Evaluación del modelo

      history['train_loss'].append(train_loss) # Pérdida de entrenamiento
      history['train_acc'].append(train_acc) # Precisión de entrenamiento
      history['val_loss'].append(val_loss) # Pérdida de validación
      history['val_acc'].append(val_acc) # Precisión de validación
      # Imprimir métricas
      print("---------------------------------------")
      print(f"Epoch {epoch+1}/{epochs}")
      print(f"Train Loss: {train_loss:.4f}, Train Acc: {100 * train_acc:.2f}%")
      print(f"Val Loss: {val_loss:.4f}, Val Acc: {100 * val_acc:.2f}%")

      # Ajustar el LR según val_loss
      old_lr = optimizer.param_groups[0]['lr'] # Guardar el LR antiguo
      scheduler.step(val_loss) # Ajustar el LR
      new_lr = optimizer.param_groups[0]['lr'] # Guardar el nuevo LR
      if new_lr != old_lr: # Si el LR ha cambiado
          print(f"Reduciendo el LR: {old_lr} -> {new_lr}") # Imprimir el cambio de LR

      # Guardar el mejor modelo
      if val_acc > best_acc: # Si la precisión de validación es mejor que la mejor precisión
          best_acc = val_acc # Actualizar la mejor precisión
          best_model_wts = copy.deepcopy(model.state_dict()) # Guardar los mejores pesos del modelo
          counter = 0 # Reiniciar el contador
          # Imprimir mensaje de éxito
          print("Mejor modelo entrenado y guardado")
      else: # Si no hay mejora
          counter += 1 # Incrementar el contador
          # Imprimir mensaje de falta de mejora
          print(f"Sin mejora ({counter}/{patience})")


      if counter >= patience: # Si el contador alcanza la paciencia(3)
          # Imprimir mensaje de early stopping
          print("Early stopping activado")
          break # Detener el entrenamiento

  # Devuelve el historial, los mejores pesos del modelo y la mejor precisión
  return history, best_model_wts, best_acc

# Llamando a la función de entrenamiento --> Fase de entrenamiento 1
training_phase1, best_model_wts, best_acc = train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs, best_acc, history)

<font color="64D7F5">--Guardar modelos entrenados en un directorio del Drive--

In [ ]:
#Montar el directorio del Google Drive en Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# Crear el directorio para guardar los modelos entrenados
save_dir = "/content/drive/MyDrive/Modelos_entrenados_ML_DL/Intel_Image_Class_PyTorch_CNN"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
import os
import torch

def save_model(model, type_model, num_classes, save_dir, phase_name="phase", best_acc=best_acc):
  """
    Guarda un modelo entrenado en formato checkpoint de PyTorch.

    Args:
        model: Modelo entrenado (nn.Module).
        type_model: Tipo de modelo (str), ej. "SimpleCNN", "ResNet18", "EfficientNetB0".
        num_classes: Número de clases del dataset.
        save_dir: Directorio donde guardar el checkpoint.
        phase_name: Nombre de la fase de entrenamiento (ej. "phase1", "phase2").
    """

  # Aplicar los mejores pesos antes de guardar
  model.load_state_dict(best_model_wts)

  # Guardar en checkpoint: tipo de modelo, clases y pesos
  checkpoint = {
      "type_model": type_model, # Tipo de modelo
      "num_classes": num_classes, # Número de clases
      "state_dict": model.state_dict(), # Pesos del modelo
      "best_acc": best_acc # Mejor precisión
  }

  # Guardar el modelo (mejor modelo)
  model_path = os.path.join(save_dir, f"{type_model}_{phase_name}.pth") # Ruta del modelo
  torch.save(checkpoint, model_path) # Guardar el modelo
  # Imprimir la mejor precisión y la ruta del modelo guardado
  print(f"Entrenamiento terminado. Mejor val_acc: {100 * best_acc:.2f}%")
  print(f"Modelo guardado en: {model_path}")

  return model_path

# Guardar el modelo de la fase 1
save_model(model, type_model, num_classes, save_dir, phase_name="phase1")

<font color="64D7F5">--Cargar los modelos entrenados y el historial--

In [ ]:
# Cargar checkpoint
checkpoint = torch.load(model_path, map_location=device)

# Reconstruir el modelo según el modelo guardado
if checkpoint["type_model"] == "ResNet18":
  model_loaded = build_ResNet18(checkpoint["num_classes"]).to(device) # Reconstruir modelo ResNet18
elif checkpoint["type_model"] == "SimpleCNN":
  model_loaded = build_SimpleCNN(checkpoint["num_classes"]).to(device) # Reconstruir modelo SimpleCNN
elif checkpoint["type_model"] == "EfficientNetB0":
  model_loaded = build_EfficientNetB0(checkpoint["num_classes"]).to(device) # Reconstruir modelo EfficientNetB0

# Cargar los pesos del modelo
model_loaded.load_state_dict(checkpoint["state_dict"])
model_loaded.eval() # Establecer el modelo en modo de evaluación

# Imprimir mensaje de éxito
print(f"Modelo {checkpoint['type_model']} cargado correctamente en el dispositivo {device}.")

<font color="64D7F5">--Evaluación en test--

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = evaluate(model_loaded, test_loader, criterion, device)
# Imprimir métricas
print(f"Test Loss: {test_loss:.4f}, Test Acc: {100 * test_acc:.2f}%")

<font color="64D7F5">--Curvas de entrenamiento--

In [ ]:
# Obtener el número de épocas entrenadas
epochs_ran = len(history['train_loss'])

# Graficar las métricas de entrenamiento(Pérdida)
plt.figure(figsize=(12, 5)) # Tamaño de la figura
plt.subplot(1, 2, 1) # Gráfica de Pérdida
plt.plot(range(1, epochs_ran + 1), history['train_loss'], label='Train Loss') # Pérdida de entrenamiento
plt.plot(range(1, epochs_ran + 1), history['val_loss'], label='Val Loss') # Pérdida de validación
plt.xlabel('Epochs') # Épocas
plt.ylabel('Loss') # Pérdida
plt.legend() # Leyenda
plt.title('Curva de Pérdida') # Título de la gráfica

# Graficar las métricas de entrenamiento(Precisión)
plt.subplot(1, 2, 2) # Gráfica de Precisión
plt.plot(range(1, epochs_ran + 1), [a*100 for a in history['train_acc']], label='Train Acc') # Precisión de entrenamiento
plt.plot(range(1, epochs_ran + 1), [a*100 for a in history['val_acc']], label='Val Acc') # Precisión de validación
plt.xlabel('Epochs') # Épocas
plt.ylabel('Accuracy (%)') # Precisión
plt.legend() # Leyenda
plt.title('Curva de Precisión') # Título de la gráfica

plt.show() # Mostrar las gráficas

<font color="64D7F5">--Matriz de confusión y reporte--

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, balanced_accuracy_score
import seaborn as sns


y_true, y_pred = [], [] # Listas para almacenar las etiquetas verdaderas y las predicciones
model.eval() # Establecer el modelo en modo de evaluación
with torch.no_grad(): # Desactivar el cálculo de gradientes
    for imgs, labels in test_loader: # Iterar sobre el conjunto de prueba
        imgs, labels = imgs.to(device), labels.to(device) # Mover imágenes y etiquetas al dispositivo
        outputs = model(imgs) # Obtener las salidas del modelo
        _, preds = outputs.max(1) # Obtener las predicciones
        y_true.extend(labels.cpu().numpy()) # Almacenar las etiquetas verdaderas
        y_pred.extend(preds.cpu().numpy()) # Almacenar las predicciones

# Imprimir un resumen de las métricas
print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=idx_to_class.values()))

# Imprimir el F1 Score y la Balanced Accuracy
print(f"F1 Score: {f1_score(y_true, y_pred, average='macro'):.2f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_true, y_pred):.2f}")

cm = confusion_matrix(y_true, y_pred) # Calcular la matriz de confusión

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6)) # Tamaño de la figura
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=idx_to_class.values(), yticklabels=idx_to_class.values()) # Mapa de calor
plt.xlabel('Predicción') # Etiqueta del eje x
plt.ylabel('Real') # Etiqueta del eje y
plt.title('Matriz de Confusión') # Título de la gráfica
plt.show() # Mostrar la gráfica

<font color="64D7F5">--Visualización de predicciones--

In [ ]:
mean = np.array(imagenet_mean)[:, None, None] # Media de ImageNet
std = np.array(imagenet_std)[:, None, None] # Desviación estándar de ImageNet

# Función para mostrar predicciones aleatorias
# n es el número de imágenes a mostrar
def show_random_predictions(model, loader, n=8):
    model.eval() # Establecer el modelo en modo de evaluación
    idxs = random.sample(range(len(loader.dataset)), n) # Seleccionar índices aleatorios
    imgs, labels = [], [] # Listas para almacenar imágenes y etiquetas
    for i in idxs: # Iterar sobre los índices seleccionados
        img, label = loader.dataset[i] # Obtener la imagen y la etiqueta
        imgs.append(img.unsqueeze(0)) # Añadir la imagen a la lista
        labels.append(label) # Añadir la etiqueta a la lista
    imgs = torch.cat(imgs).to(device) # Concatenar las imágenes y mover al dispositivo
    labels = torch.tensor(labels).to(device) # Mover las etiquetas al dispositivo

    outputs = model(imgs) # Obtener las salidas del modelo
    _, preds = outputs.max(1) # Obtener las predicciones

    # Graficar las predicciones
    plt.figure(figsize=(15, 6)) # Tamaño de la figura
    for i in range(n): # Iterar sobre las imágenes
        img = imgs[i].cpu().numpy() # Convertir la imagen a un array de NumPy
        img = np.transpose(img, (1, 2, 0)) # Transponer las dimensiones de la imagen
        img = img * std.transpose(1, 2, 0) + mean.transpose(1, 2, 0) # Desnormalizar la imagen
        img = np.clip(img, 0, 1) # Asegurarse de que los valores estén en el rango [0, 1]

        # Determinar el color del título según la predicción
        color = "green" if preds[i].item() == labels[i].item() else "red" # green=correcto, red=incorrecto
        plt.subplot(2, n//2, i+1) # Crear un subplot para cada imagen
        plt.imshow(img) # Mostrar la imagen
        plt.axis("off") # Ocultar los ejes
        # Mostrar la etiqueta real y la predicción dependiendo del color
        plt.title(f"Pred: {idx_to_class[preds[i].item()]}\nReal: {idx_to_class[labels[i].item()]}", color=color)
    plt.show() # Mostrar la figura

# Mostrar predicciones aleatorias
show_random_predictions(model, test_loader)

<font color="64D7F5">--Visualización de errores más comunes(por clases)--

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, balanced_accuracy_score
import seaborn as sns
from collections import Counter

def show_misclassified_images(model, loader, n=12):
    model.eval() # Establecer el modelo en modo de evaluación
    misclassified = [] # Lista para almacenar las imágenes clasificadas incorrectamente

    with torch.no_grad(): # Desactivar el cálculo de gradientes
        for imgs, labels in loader: # Iterar sobre el DataLoader
            imgs, labels = imgs.to(device), labels.to(device) # Mover imágenes y etiquetas al dispositivo
            outputs = model(imgs) # Obtener las salidas del modelo
            _, preds = outputs.max(1) # Obtener las predicciones

            # Guardar índices de errores
            for i in range(len(labels)):
                if preds[i] != labels[i]:
                    misclassified.append((imgs[i].cpu(), preds[i].item(), labels[i].item()))

        # Imprimir el número de errors
        print(f"Total de errores: {len(misclassified)}")

        if len(misclassified) == 0: # Si no hay errores
            print("No hay errores en este conjunto.")
            return

        # Seleccionar aleatoriamente n errores
        sample_errors = random.sample(misclassified, min(n, len(misclassified)))

        # Graficar los errores más comunes
        plt.figure(figsize=(15, 8)) # Tamaño de la figura
        for i, (img_tensor, pred, label) in enumerate(sample_errors):
            img = img_tensor.numpy().transpose(1, 2, 0) # Transponer las dimensiones de la imagen
            img = img * std.transpose(1, 2, 0) + mean.transpose(1, 2, 0) # Desnormalizar la imagen
            img = np.clip(img, 0, 1) # Asegurarse de que los valores estén en el rango [0, 1]

            plt.subplot(3, n//3, i+1) # Crear un subplot para cada imagen
            plt.imshow(img) # Mostrar la imagen
            plt.axis("off") # Ocultar los ejes
            plt.title(f"Pred: {idx_to_class[pred]}\nReal: {idx_to_class[label]}", color="red") # Mostrar la etiqueta predecida y la real

        plt.suptitle("Errores más comunes por clase", fontsize=14) # Título de la gráfica
        plt.show() # Mostrar la figura

# Usando la función creada
show_misclassified_images(model, test_loader)

<font color="64D7F5">--Encontrar las confusiones más freqüentes--

In [ ]:
def top_confusions(y_true, y_pred, idx_to_class, top_k=5):
  cm = confusion_matrix(y_true, y_pred) # Calcular la matriz de confusión
  confusions = [] # Lista para almacenar las confusiones

  for i in range(len(cm)):
    for j in range(len(cm)):
      if i != j and cm[i][j] > 0: # ignoramos los aciertos (diagonal)
        confusions.append((cm[i][j], idx_to_class[i], idx_to_class[j]))

  confusions.sort(reverse=True, key=lambda x: x[0]) # Ordenar por frecuencia
  print(f"Top {top_k} confusiones más frecuentes:")
  for n, (count, real, pred) in enumerate(confusions[:top_k], 1):
    print(f"{n}. Real: {real} --> Pred: {pred} ({count} veces)")

  return confusions[:top_k] # Retornar las confusiones más frecuentes

# Usando la función creada
freq_confusions = top_confusions(y_true, y_pred, idx_to_class)

<font color="64D7F5">--Mejora en el DataAugmentation dependiendo el tipo de confusión--

---


<font color="64D7F5">**Mountain ↔ Glacier:** Se confunden porque ambas tienen texturas parecidas (blanco, rocas, nieve).

<font color="64D7F5">➜ Augmentations con color jitter (brillo/contraste) ayudan a que el modelo aprenda a diferenciar “montaña con nieve” de “glaciar puro”. Rotaciones también, porque ángulos diferentes no deberían confundir.

---


<font color="64D7F5">**Buildings ↔ Street:**
<font color="64D7F5">Se confunden porque las calles suelen tener edificios.

<font color="64D7F5">➜ Distorsiones de perspectiva y rotaciones pequeñas ayudan a que el modelo aprenda que un edificio no siempre implica “street”.

---


<font color="64D7F5">**Glacier ↔ Sea:**
<font color="64D7F5">Se confunden porque ambos pueden ser azulados y con reflejos.

<font color="64D7F5">➜ Augmentations con saturación y brillo ayudan a que el modelo aprenda a distinguir “azul sólido de agua” de “azul con hielo”.



In [ ]:
# Funcion que ugiere técnicas de data augmentation según las confusiones más comunes.
# confusions = lista [(count, real_class, pred_class), ...]
def suggest_augmentations(top_confusions):
    """

    """
    suggestions = {} # Diccionario para almacenar sugerencias

    # Iterar sobre las confusiones más comunes
    for _, real, pred in top_confusions:
      if (real, pred) in [('mountain','glacier'), ('glacier','mountain')]:
        # Sugerencias para montañas y glaciares
        suggestions[real] = [
          "RandomResizedCrop", "RandomRotation(15°)", "ColorJitter(brightness/contrast)",
                "RandomHorizontalFlip", "RandomVerticalFlip", "Zoom/Scale variations"
            ]
      elif (real, pred) in [('buildings','street'), ('street','buildings')]:
          # Sugerencias para edificios y calles
          suggestions[real] = [
              "RandomRotation(10°)", "RandomPerspective", "ColorJitter", "RandomHorizontalFlip"
            ]
      elif (real, pred) in [('glacier','sea'), ('sea','glacier')]:
          # Sugerencias para glaciares y mares
          suggestions[real] = [
              "ColorJitter(saturation/brightness)", "RandomRotation(10°)", "RandomResizedCrop"
            ]
      else:
            # Sugerencias para clases sin confusión frecuente
            suggestions[real] = [
              "RandomResizedCrop", "RandomHorizontalFlip", "ColorJitter"
            ]

    print("Sugerencias de Data Augmentation según las confusiones más comunes:")
    # Imprimir las sugerencias
    for cls, aug in suggestions.items():
        print(f"- {cls.capitalize()}: {', '.join(aug)}")

    # Devolver las sugerencias
    return suggestions

# Obtener las sugerencias de data augmentation
suggested_augmentations = suggest_augmentations(freq_confusions)

<font color="64D7F5">--Aplicando el Data Augmentando según las confusiones obtenidas--

In [ ]:
import torchvision.transforms as transforms

def get_classwise_transforms(top_confusions, input_size=224):
    """
    Crea un diccionario con transforms.Compose por clase
    basado en las confusiones más frecuentes.

    Args:
        top_confusions: lista de tuplas (count, real_class, pred_class)
        input_size: tamaño final al que se redimensionarán las imágenes
    """
    class_transforms = {}

    for _, real, pred in top_confusions:
        if (real, pred) in [('mountain','glacier'), ('glacier','mountain')]:
            # Augmentations específicas para montañas y glaciares
            aug = transforms.Compose([
                transforms.RandomResizedCrop(input_size, scale=(0.85, 1.0)), # Recorte aleatorio
                transforms.RandomRotation(15), # Rotación aleatoria
                transforms.ColorJitter(brightness=0.2, contrast=0.2), # Variación de color
                transforms.RandomHorizontalFlip(p=0.5), # Flip horizontal
                transforms.ToTensor(), # Convertir a tensor
                transforms.Normalize(imagenet_mean, imagenet_std) # Normalización
            ])
            # Asignar las transformaciones a la clase correspondiente
            class_transforms[real] = aug

        elif (real, pred) in [('buildings','street'), ('street','buildings')]:
            # Augmentations específicas para edificios y calles
            aug = transforms.Compose([
                transforms.RandomResizedCrop(input_size, scale=(0.9, 1.0)), # Recorte aleatorio
                transforms.RandomRotation(10), # Rotación aleatoria
                transforms.RandomPerspective(distortion_scale=0.2, p=0.5), # Perspectiva aleatoria
                transforms.RandomHorizontalFlip(p=0.5), # Flip horizontal
                transforms.ToTensor(), # Convertir a tensor
                transforms.Normalize(imagenet_mean, imagenet_std) # Normalización
            ])
            # Asignar las transformaciones a la clase correspondiente
            class_transforms[real] = aug

        elif (real, pred) in [('glacier','sea'), ('sea','glacier')]:
            # Augmentations específicas para glaciares y mares
            aug = transforms.Compose([
                transforms.RandomResizedCrop(input_size, scale=(0.85, 1.0)), # Recorte aleatorio
                transforms.RandomRotation(10), # Rotación aleatoria
                transforms.ColorJitter(saturation=0.3, brightness=0.2), # Variación de color
                transforms.ToTensor(), # Convertir a tensor
                transforms.Normalize(imagenet_mean, imagenet_std) # Normalización
            ])
            # Asignar las transformaciones a la clase correspondiente
            class_transforms[real] = aug

        else:
            # Default augmentation para clases sin confusión frecuente
            aug = transforms.Compose([
                transforms.Resize((img_size)), # Redimensionar
                transforms.RandomHorizontalFlip(p=0.5), # Flip horizontal
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1), # Variación de color
                transforms.ToTensor(), # Convertir a tensor
                transforms.Normalize(imagenet_mean, imagenet_std) # Normalización
            ])
            # Asignar las transformaciones a la clase correspondiente
            class_transforms[real] = aug

    # Devolver las transformaciones por clase
    return class_transforms

# Obtener las transformaciones personalizadas por clase
classwise_transforms = get_classwise_transforms(freq_confusions)

<font color="64D7F5">--Aplicar el Custom Data Augmentation al Dataset--

In [ ]:
from torchvision.datasets import ImageFolder

# Dataset personalizado con transformaciones específicas por clase
class CustomDataset(ImageFolder):
    # Inicialización del dataset
    def __init__(self, root, classwise_transforms, default_transform, apply_prob=0.7, *args, **kwargs):
        super().__init__(root, transform=None, *args, **kwargs) # Llamar al constructor de la clase base
        self.classwise_transforms = classwise_transforms # Transformaciones específicas por clase
        self.default_transform = default_transform # Transformación por defecto
        self.apply_prob = apply_prob # Probabilidad de aplicar la transformación

    # Obtener un elemento del dataset
    def __getitem__(self, index):
        path, target = self.samples[index] # Obtener la ruta y el objetivo
        sample = self.loader(path) # Cargar la imagen
        class_name = self.classes[target] # Obtener el nombre de la clase
        if class_name in self.classwise_transforms and random.random() < self.apply_prob: # Aplicar la transformación con probabilidad
          transform = self.classwise_transforms[class_name] # Obtener la transformación correspondiente
        else:
          transform = self.default_transform # Usar la transformación por defecto

        # Aplicar la transformación
        if transform is not None:
            sample = transform(sample)

        # Devolver la muestra y el objetivo
        return sample, target

<font color="64D7F5">--Cambiando la entrada del Dataset para que se aplique el Custom Data Augmentation en las clases que se encuentran las confusiones--

In [ ]:
# Dataset con augmentations por clase
custom_train_ds = CustomDataset(
    root=train_dir, # Directorio de entrenamiento
    classwise_transforms=classwise_transforms, # Transformaciones específicas por clase
    default_transform=train_tfms, # Transformación por defecto
    apply_prob=0.7, # Probabilidad de aplicar la transformación
)

# DataLoader para el conjunto de entrenamiento
train_loader_phase2 = DataLoader(custom_train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

# Fase de entrenamiento 2
history_phase2 = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

#Reiniciando variables
counter = 0 # Reiniciar early stopping
best_acc = 0.0 # Nueva variable para alamacenar el best_acc de la Fase 2

training_phase2, best_model_wts, best_acc_phase2 = train_model(model, train_loader_phase2, val_loader, criterion, optimizer, device, epochs, best_acc, history_phase2)

<font color="64D7F5">--Guardado del modelo entrenado en la Fase 2--

In [ ]:
# Guardar el modelo de la fase 1
save_model(model, type_model, num_classes, save_dir, phase_name="phase2", best_acc=best_acc_phase2)

<font color="64D7F5">--Comparativa entre las dos fases de entrenamiento--

In [ ]:
print("Comparativa de las dos fases de entrenamiento:")
print(f"Fase 1: Mejor precisión: {100 * best_acc:.2f}%")
print(f"Fase 2: Mejor precisión: {100 * best_acc_phase2:.2f}%")

<font color="64D7F5">--Comparativa grafica entre las dos fases de entrenamiento--

In [ ]:
import matplotlib.pyplot as plt

# Función para graficar la historia de entrenamiento
def training_history(history1, history2, label1="Fase 1", label2="Fase 2"):
  epochs1 = range(1, len(history1['train_loss']) + 1) # Épocas de la fase 1
  epochs2 = range(1, len(history2['train_loss']) + 1) # Épocas de la fase 2

  plt.figure(figsize=(12, 5)) # Tamaño de la figura

  # Pérdida
  plt.subplot(1, 2, 1)
  plt.plot(epochs1, history1['train_loss'], label=f'{label1} Train Loss') # Pérdida de entrenamiento fase 1
  plt.plot(epochs1, history1['val_loss'], label=f'{label1} Val Loss') # Pérdida de validación fase 1
  plt.plot(epochs2, history2['train_loss'], label=f'{label2} Train Loss') # Pérdida de entrenamiento fase 2
  plt.plot(epochs2, history2['val_loss'], label=f'{label2} Val Loss') # Pérdida de validación fase 2
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.title('Evolución de la Pérdida')
  plt.grid(True)

  # Exactitud
  plt.subplot(1, 2, 2)
  plt.plot(epochs1, history1["train_acc"], "b-", label=f"{label1} Train Acc") # Exactitud de entrenamiento fase 1
  plt.plot(epochs1, history1["val_acc"], "b--", label=f"{label1} Val Acc") # Exactitud de validación fase 1
  plt.plot(epochs2, history2["train_acc"], "g-", label=f"{label2} Train Acc") # Exactitud de entrenamiento fase 2
  plt.plot(epochs2, history2["val_acc"], "g--", label=f"{label2} Val Acc") # Exactitud de validación fase 2
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy (%)")
  plt.legend()
  plt.title("Evolución de la Exactitud")
  plt.grid(True)


  plt.show()

# Graficar la historia de entrenamiento combinada
training_history(training_phase1, training_phase2)

<font color="64D7F5">--Guardar el historial de entrenamiento--

In [ ]:
# Guardar el historial
import json
history_path = os.path.join(save_dir, "history.json") # Ruta del historial
with open(history_path, "w") as f: # Abrir el archivo en modo escritura
    json.dump(history, f) # Guardar el historial en formato JSON
# Mostrar mensaje de confirmación
print(f"Historial guardado en: {history_path}")

<font color="64D7F5">--Cargar el historial de entrenamiento--

In [ ]:
# Cargar el historial
with open(history_path, "r") as f: # Abrir el archivo en modo lectura
    history = json.load(f) # Cargar el historial desde el archivo JSON
# Mostrar mensaje de confirmación
print("Historial cargado correctamente.")